In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [9]:
df1 = pd.read_csv('df1_modified.csv')
df2 = pd.read_csv('df2_modified.csv')
df3 = pd.read_csv('df3_modified.csv')
df4 = pd.read_csv('df4_modified.csv')
df5 = pd.read_csv('df5_modified.csv')
df6 = pd.read_csv('df6_modified.csv')
df_concat = pd.concat([df1, df2, df3,df4,df5,df6], axis=0, ignore_index=True)
df_concat

,underlying_symbol,quote_datetime,root,expiration,strike,option_type,open,high,low,close,...,implied_volatility,delta,gamma,theta,vega,rho,open_interest,risk_free_rate,time_to_maturity,option_price
0,^SPX,2024-07-26 10:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,5.4324,0.9995,0.0000,0.0,0.0314,-0.0047,24,5.49,0.083333,5250.00
1,^SPX,2024-07-26 11:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,5.0932,0.9997,0.0000,0.0,0.0202,-0.0117,24,5.49,0.083333,5245.90
2,^SPX,2024-07-26 12:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,4.9448,0.9998,0.0000,0.0,0.0160,-0.0184,24,5.49,0.083333,5269.65
3,^SPX,2024-07-26 13:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,4.8580,1.0000,0.0000,0.0,0.0141,-0.0210,24,5.49,0.083333,5284.75
4,^SPX,2024-07-26 14:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,4.2050,1.0000,0.0000,0.0,0.0034,-0.0269,24,5.49,0.083333,5258.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021309,^SPX,2024-10-25 12:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1264,-0.9829,0.0001,0.0,3.4015,-68.8414,1,3.98,1.349206,1682.65
1021310,^SPX,2024-10-25 13:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1303,-0.9811,0.0001,0.0,3.5962,-68.7475,1,3.98,1.349206,1705.00
1021311,^SPX,2024-10-25 14:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1331,-0.9790,0.0001,0.0,3.8282,-68.6286,1,3.98,1.349206,1711.15
1021312,^SPX,2024-10-25 15:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1304,-0.9817,0.0001,0.0,3.5291,-68.7458,1,3.98,1.349206,1712.45


In [10]:
# Black-Scholes model for Call and Put options
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    if K == 0 or T == 0 or sigma == 0:
        return np.nan  # Return NaN for invalid values
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2)

def BS_PUT(S, K, T, r, sigma):
    if K == 0 or T == 0 or sigma == 0:
        return np.nan  # Return NaN for invalid values
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * N(-d2) - S * N(-d1)

# Apply the Black-Scholes formula to calculate option prices for each row in the dataset
def calculate_option_price(row):
    S = row['active_underlying_price']
    K = row['strike']
    T = row['time_to_maturity']
    r = row['risk_free_rate']/100  # Ensure risk-free rate is in decimal form (not percentage)
    sigma = row['implied_volatility'] / 100  # Ensure volatility is in decimal form (not percentage)

    if row['option_type'] == 'C':  # Call option
        return BS_CALL(S, K, T, r, sigma)
    elif row['option_type'] == 'P':  # Put option
        return BS_PUT(S, K, T, r, sigma)

# Calculate the option price and add it to a new column in the dataframe
df_concat['BS_calculated_option_price'] = df_concat.apply(calculate_option_price, axis=1)

In [ ]:
#df_concat.to_csv('df_concat_withBSMprice.csv', index=False)

In [13]:

# Filter rows where neither BS_calculated_option_price nor option_price is NaN, and the calculated price is not 0
valid_df = df_concat.dropna(subset=['BS_calculated_option_price', 'option_price'])
valid_df = valid_df[valid_df['BS_calculated_option_price'] != 0]

# Get the actual option price and the calculated BS option price
actual_prices = valid_df['option_price']
calculated_prices = valid_df['BS_calculated_option_price']

# Calculate Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
mse = mean_squared_error(actual_prices, calculated_prices)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(actual_prices, calculated_prices)

# Calculate R-squared (R2)
r_squared = r2_score(actual_prices, calculated_prices)

# Display results
rmse, mae, r_squared

(np.float64(111.8825657474548),
 np.float64(61.72062937374769),
 0.9866680498275879)